In [41]:
import pandas
import numpy
import datetime

sales = pandas.read_csv("Data Model - Pizza Sales.csv")

In [42]:
sales.columns

Index(['order_details_id', 'order_id', 'pizza_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name'],
      dtype='object')

In [43]:
for column_id in sales.columns:
    print(sales[column_id].unique())

[    1     2     3 ... 48618 48619 48620]
[    1     2     3 ... 21348 21349 21350]
['hawaiian_m' 'classic_dlx_m' 'five_cheese_l' 'ital_supr_l' 'mexicana_m'
 'thai_ckn_l' 'ital_supr_m' 'prsc_argla_l' 'bbq_ckn_s' 'the_greek_s'
 'spinach_supr_s' 'classic_dlx_s' 'green_garden_s' 'ital_cpcllo_l'
 'ital_supr_s' 'mexicana_s' 'spicy_ital_l' 'spin_pesto_l' 'veggie_veg_s'
 'mexicana_l' 'southw_ckn_l' 'bbq_ckn_l' 'cali_ckn_l' 'cali_ckn_m'
 'pepperoni_l' 'cali_ckn_s' 'ckn_pesto_l' 'big_meat_s' 'soppressata_l'
 'four_cheese_l' 'napolitana_s' 'calabrese_m' 'four_cheese_m'
 'ital_veggie_s' 'mediterraneo_m' 'peppr_salami_s' 'spinach_fet_l'
 'napolitana_l' 'sicilian_l' 'ital_cpcllo_m' 'southw_ckn_s' 'bbq_ckn_m'
 'pepperoni_m' 'prsc_argla_s' 'sicilian_m' 'veggie_veg_l' 'ckn_alfredo_s'
 'pepperoni_s' 'green_garden_l' 'green_garden_m' 'pep_msh_pep_l'
 'hawaiian_s' 'peppr_salami_m' 'ckn_alfredo_m' 'peppr_salami_l'
 'spin_pesto_s' 'thai_ckn_m' 'classic_dlx_l' 'ckn_pesto_m' 'the_greek_xl'
 'hawaiian_l' 'pep

In [44]:
#convert order dates to a nicer format
def changeDateFormat(date: str) -> str:
    datetime_obj = datetime.datetime.strptime(date, "%m/%d/%Y")
    return datetime_obj.strftime("%Y-%m-%d")

sales["order_date"] = sales['order_date'].apply(changeDateFormat)

#convert order dates to an indexed list for graphing/counting purposes
def dateToIndex(date: str) -> int:
    datetime_obj = datetime.datetime.strptime(date, "%Y-%m-%d")
    return datetime_obj.timetuple().tm_yday

def dateToWeekday(date: str) -> str:
    datetime_obj = datetime.datetime.strptime(date, "%Y-%m-%d")
    return ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"][datetime_obj.weekday()]

sales["order_date_index"] = sales["order_date"].apply(dateToIndex)
sales["order_date_weekday"] = sales["order_date"].apply(dateToWeekday)

In [45]:
# build a dataframe that coalesces individual sales into their respective orders
orders = pandas.DataFrame(columns = ["item_count", "unique_item_count", "total_price", "order_date_index", "order_date", "order_date_weekday", "order_time"])

order_row = None
order_id = None
for row_index in sales.index:
    item = sales.loc[row_index]
    if order_row is None or order_id != item["order_id"]:
        if not order_row is None:
            orders.loc[order_id] = order_row

        order_id = item["order_id"]
        order_row = {
            "item_count": 0,
            "unique_item_count": 0,
            "total_price": 0,
            "order_date_index": item["order_date_index"],
            "order_date": item["order_date"],
            "order_date_weekday": item["order_date_weekday"],
            "order_time": item["order_time"]
        }
    order_row["item_count"] += item["quantity"]
    order_row["unique_item_count"] += 1
    order_row["total_price"] += item["total_price"]
orders.loc[order_id] = order_row

orders

,item_count,unique_item_count,total_price,order_date_index,order_date,order_date_weekday,order_time
1,1,1,13.25,1,2015-01-01,Thursday,11:38:36
2,5,5,92.00,1,2015-01-01,Thursday,11:57:40
3,2,2,37.25,1,2015-01-01,Thursday,12:12:28
4,1,1,16.50,1,2015-01-01,Thursday,12:16:31
5,1,1,16.50,1,2015-01-01,Thursday,12:21:30
...,...,...,...,...,...,...,...
21346,4,4,62.25,365,2015-12-31,Thursday,20:51:07
21347,4,4,66.50,365,2015-12-31,Thursday,21:14:37
21348,3,3,46.70,365,2015-12-31,Thursday,21:23:10
21349,1,1,20.25,365,2015-12-31,Thursday,22:09:54


In [46]:
#create a separate dataframe to help count daily statistics
daily_sales = pandas.DataFrame(columns = ["order_count", "item_count", "revenue", "date", "weekday"])

day_row = None
order_date_index = None
for row_index in orders.index:
    order = orders.loc[row_index]
    if day_row is None or order_date_index != order["order_date_index"]:
        if not day_row is None:
            daily_sales.loc[order_date_index] = day_row

        order_date_index = order["order_date_index"]
        day_row = {
            "item_count": 0,
            "order_count": 0,
            "revenue": 0,
            "date": order["order_date"],
            "weekday": order["order_date_weekday"]
        }
    day_row["item_count"] += order["item_count"]
    day_row["order_count"] += 1
    day_row["revenue"] += order["total_price"]
daily_sales.loc[order_date_index] = day_row

#remove all the days with no sales
daily_sales = daily_sales.drop(daily_sales.index[daily_sales["item_count"] == 0])

daily_sales

,order_count,item_count,revenue,date,weekday
1,69,162,2713.85,2015-01-01,Thursday
2,67,165,2731.90,2015-01-02,Friday
3,66,158,2662.40,2015-01-03,Saturday
4,52,106,1755.45,2015-01-04,Sunday
5,54,125,2065.95,2015-01-05,Monday
...,...,...,...,...,...
361,35,89,1419.00,2015-12-27,Sunday
362,39,102,1637.20,2015-12-28,Monday
363,27,80,1353.25,2015-12-29,Tuesday
364,32,82,1337.80,2015-12-30,Wednesday


In [50]:
# make a dataframe to keep track of statistics on the types of pizza
pizza_properties = pandas.DataFrame(columns = ["price_S", "price_M", "price_L", "price_XL", "price_XXL", "ingredients", "quantity_sold", "category"])

for pizza_name in sales["pizza_name"].unique():
    pizza_sales = sales[sales["pizza_name"] == pizza_name]
    pizza_row = {
        "price_S": numpy.nan,
        "price_M": numpy.nan,
        "price_L": numpy.nan,
        "price_XL": numpy.nan,
        "price_XXL": numpy.nan,
        "ingredients": pizza_sales.iloc[0]["pizza_ingredients"],
        "quantity_sold": 0,
        "category": pizza_sales.iloc[0]["pizza_category"]
    }
    for sale_index in pizza_sales.index:
        sale = pizza_sales.loc[sale_index]
        pizza_row["price_%s" %(sale["pizza_size"])] = sale["unit_price"]
        pizza_row["quantity_sold"] += sale["quantity"]
    pizza_properties.loc[pizza_name] = pizza_row

pizza_properties

,price_S,price_M,price_L,price_XL,price_XXL,ingredients,quantity_sold,category
The Hawaiian Pizza,10.50,13.25,16.50,NaN,NaN,"Sliced Ham, Pineapple, Mozzarella Cheese",2422,Classic
The Classic Deluxe Pizza,12.00,16.00,20.50,NaN,NaN,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",2453,Classic
The Five Cheese Pizza,NaN,NaN,18.50,NaN,NaN,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",1409,Veggie
The Italian Supreme Pizza,12.50,16.50,20.75,NaN,NaN,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",1884,Supreme
The Mexicana Pizza,12.00,16.00,20.25,NaN,NaN,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",1484,Veggie
The Thai Chicken Pizza,12.75,16.75,20.75,NaN,NaN,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha...",2371,Chicken
The Prosciutto and Arugula Pizza,12.50,16.50,20.75,NaN,NaN,"Prosciutto di San Daniele, Arugula, Mozzarella...",1457,Supreme
The Barbecue Chicken Pizza,12.75,16.75,20.75,NaN,NaN,"Barbecued Chicken, Red Peppers, Green Peppers,...",2432,Chicken
The Greek Pizza,12.00,16.00,20.50,25.5,35.95,"Kalamata Olives, Feta Cheese, Tomatoes, Garlic...",1420,Classic
The Spinach Supreme Pizza,12.50,16.50,20.75,NaN,NaN,"Spinach, Red Onions, Pepperoni, Tomatoes, Arti...",950,Supreme


In [48]:
# make another dataframe to keep track of ingredients, which pizzas they are in, and how many pizzas are sold with those ingredients
ingredient_set = set()
for ingredients_list in pizza_properties["ingredients"]:
    ingredient_set.update(ingredients_list.split(", "))
ingredient_set

ingredient_properties = pandas.DataFrame(columns = ["quantity_sold", "pizza_types_containing"])

for ingredient in ingredient_set:
    unique_pizzas = set()
    for pizza_name in pizza_properties.index:
        if ingredient in pizza_properties.loc[pizza_name]["ingredients"]:
            unique_pizzas.add(pizza_name)
    sales_with_this_ingredient = sales[sales["pizza_name"].apply(lambda pizza_name: pizza_name in unique_pizzas)]
    ingredient_properties.loc[ingredient] = [sales_with_this_ingredient["quantity"].sum(), str(unique_pizzas).rstrip("}").lstrip("{")]

ingredient_properties

,quantity_sold,pizza_types_containing
Pepperoni,10540,"'The Pepperoni, Mushroom, and Peppers Pizza', ..."
Bacon,4367,"'The Classic Deluxe Pizza', 'The Big Meat Pizza'"
Calabrese Salami,1884,'The Italian Supreme Pizza'
Soppressata Salami,961,'The Soppressata Pizza'
Asiago Cheese,3383,"'The Chicken Alfredo Pizza', 'The Pepper Salam..."
...,...,...
Kalamata Olives,3304,"'The Mediterranean Pizza', 'The Greek Pizza', ..."
Anchovies,1464,'The Napolitana Pizza'
Coarse Sicilian Salami,1938,'The Sicilian Pizza'
Barbecued Chicken,2432,'The Barbecue Chicken Pizza'


In [51]:
sales.to_csv("sales.csv")
daily_sales.to_csv("daily_sales.csv")
orders.to_csv("orders.csv")
pizza_properties.to_csv("pizza_properties.csv")
ingredient_properties.to_csv("ingredient_properties.csv")